In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)
from dataclasses import dataclass

In [ ]:
#import libs to resize and read files
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
from pathlib import Path

In [ ]:
#######################################
# List all files in the folder
#######################################
files_x = list(map(str, filter(lambda file: file.is_file(), Path("C:/temp/AR_out").rglob('*'))))

In [ ]:
image_resized_list = []
file_names_list = []
#######################################
# resize all files
#######################################
for file in files_x:
    # read image
    img = cv2.imread(file, cv2.IMREAD_UNCHANGED)

    # Set Scale
    scale_percent = 10 
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dimension = (width, height)

    # resize image
    resized = cv2.resize(img, dimension, interpolation = cv2.INTER_AREA)

    #save an array
    image_resized_list.append(resized)
    file_names_list.append(file.split('\\')[-1].split('.')[0])
    
 

In [ ]:
# Convert the list to a numpy array
image_resized_array = np.array(image_resized_list)

# Get the number of images and the dimensions of each image
num_images, height, width, num_channels = image_resized_array.shape

# Reshape the array into a 2D array where each row is a vectorized image
data_2d = image_resized_array.reshape(num_images, -1)  # -1 is a placeholder that tells numpy to compute the size of this dimension

In [ ]:
mean_vector = np.mean(data_2d, axis=0)

In [ ]:
# Center the data by subtracting the mean
centered_data = data_2d - mean_vector

In [ ]:
covariance_matrix = np.dot(centered_data.T, centered_data) / (num_images - 1)

In [ ]:
covariance_matrix = np.cov(data_2d, rowvar=False)


import matplotlib.pyplot as plt
import seaborn as sns  # Optional, for better visualization

# Display a heatmap of the covariance matrix
plt.figure(figsize=(10, 10))
sns.heatmap(covariance_matrix, cmap='coolwarm', fmt='.2f')
plt.show()

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

In [ ]:
# Get the indices that would sort the eigenvalues in descending order
sorted_indices = np.argsort(eigenvalues)[::-1]

# Sort the eigenvalues and eigenvectors
sorted_eigenvalues = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:, sorted_indices]


In [ ]:
total_variance = np.sum(sorted_eigenvalues)

In [ ]:
cumulative_variance = np.cumsum(sorted_eigenvalues)

In [ ]:
num_eigenvectors_to_retain = np.argmax(cumulative_variance / total_variance >= 0.99) + 1
print (num_eigenvectors_to_retain)
#num_eigenvectors_to_retain = 500

In [ ]:
# Select the top eigenvectors to retain 99% of the variance
top_eigenvectors = sorted_eigenvectors[:, :num_eigenvectors_to_retain]


# Project the centered data onto the selected eigenvectors
projected_data = np.dot(centered_data, top_eigenvectors)

In [ ]:
# Project the centered data onto the selected eigenvectors
projected_data = np.dot(centered_data, top_eigenvectors)

In [ ]:
##############################################################################
#Making a test with a orignal, resized and projected image
##############################################################################

In [ ]:
index = 2034  # Index of the image you want to select
source_image = cv2.imread(files_x[index], cv2.IMREAD_UNCHANGED) 
resized_image = image_resized_array[index]
original_image = image_resized_array[index]

In [ ]:
projected_representation = projected_data[index]
reconstructed_image_data = np.dot(projected_representation, top_eigenvectors.T)

In [ ]:
reconstructed_image = reconstructed_image_data.reshape(height, width, num_channels)
reconstructed_image = np.clip(reconstructed_image, 0, 255).astype(np.uint8)


In [ ]:
plt.figure(figsize=(15, 5))  # Adjust the figure size to accommodate three images

# Display source image
plt.subplot(1, 3, 1)  # 1 row, 3 columns, 1st subplot
plt.imshow(cv2.cvtColor(source_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB if necessary
plt.title('Source Image')

# Display original image
plt.subplot(1, 3, 2)  # 1 row, 3 columns, 2nd subplot
plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB if necessary
#axs[1].imshow(resized_image)
plt.title('resized Image')

# Display reconstructed image
plt.subplot(1, 3, 3)  # 1 row, 3 columns, 3rd subplot
plt.imshow(cv2.cvtColor(reconstructed_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB if necessary
plt.title('Vector Image remapped')

plt.show()

##### Merge projected_data with labels

In [ ]:
import scr_deep_learning_cnn_keras_py as keras_ml
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Splitting each string by hyphen and storing the results in a new list
split_parts = [name.split('-') for name in file_names_list]

# Creating a DataFrame from the list of split parts
df02 = pd.DataFrame(split_parts, columns=['gender', 'seq', 'type'])

print(df02)

In [ ]:
projected_df = pd.DataFrame(projected_data)


In [ ]:
combined_df = pd.concat([projected_df, df02], axis=1)


In [ ]:
combined_df.head(10)

In [ ]:
from sklearn.model_selection import train_test_split

X = df[['Variance', 'Skewness', 'Curtosis', 'Entropy']]
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [ ]:
# Load the dataset (CIFAR-10)
(X_train, y_train), (X_test, y_test) = keras_ml.cifar10.load_data()

print('Learning set:')
print('X: ', end='')
print(X_train.shape)
print('y: ', end='')
print(y_train.shape)

print('Test set:')
print('X: ', end='')
print(X_test.shape)
print('y: ', end='')
print(y_test.shape)

# DEBUG. Plot some of the images
plt.figure(figsize=(18, 9))

num_rows = 4
num_cols = 5

# plot each of the images in the batch and the associated ground truth labels.
for i in range(num_rows*num_cols):
    ax = plt.subplot(num_rows, num_cols, i + 1)
    plt.imshow(X_train[i,:,:])
    ax.title.set_text(y_train[i,0])
    plt.axis("off")

# Normalize images to the range [0, 1].
X_train = X_train.astype("float32") / 255
X_test  = X_test.astype("float32") / 255

# Change the labels from integer to categorical data.
print('Original (integer) label for the first training sample: ', y_train[0])

# Convert labels to one-hot encoding.
y_train = keras_ml.to_categorical(y_train)
y_test  = keras_ml.to_categorical(y_test)

print('After conversion to categorical one-hot encoded labels: ', y_train[0])

# Create the model.
model = keras_ml.cnn_model()
model.summary()

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'],
             )

history = model.fit(X_train,
                    y_train,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_split=.3,
                   )


# Retrieve training results.
train_loss = history.history["loss"]
train_acc  = history.history["accuracy"]
valid_loss = history.history["val_loss"]
valid_acc  = history.history["val_accuracy"]

keras_ml.plot_results([ train_loss, valid_loss ],
            ylabel="Loss",
            ylim = [0.0, 5.0],
            metric_name=["Training Loss", "Validation Loss"],
            color=["g", "b"]);

keras_ml.plot_results([ train_acc, valid_acc ],
            ylabel="Accuracy",
            ylim = [0.0, 1.0],
            metric_name=["Training Accuracy", "Validation Accuracy"],
            color=["g", "b"])

In [ ]:
# Load the dataset (CIFAR-10)
(X_train, y_train), (X_test, y_test) = keras_ml.cifar10.load_data()

print('Learning set:')
print('X: ', end='')
print(X_train.shape)
print('y: ', end='')
print(y_train.shape)

print('Test set:')
print('X: ', end='')
print(X_test.shape)
print('y: ', end='')
print(y_test.shape)

# DEBUG. Plot some of the images
plt.figure(figsize=(18, 9))

num_rows = 4
num_cols = 5

# plot each of the images in the batch and the associated ground truth labels.
for i in range(num_rows*num_cols):
    ax = plt.subplot(num_rows, num_cols, i + 1)
    plt.imshow(X_train[i,:,:])
    ax.title.set_text(y_train[i,0])
    plt.axis("off")

# Normalize images to the range [0, 1].
X_train = X_train.astype("float32") / 255
X_test  = X_test.astype("float32") / 255

# Change the labels from integer to categorical data.
print('Original (integer) label for the first training sample: ', y_train[0])

# Convert labels to one-hot encoding.
y_train = keras_ml.to_categorical(y_train)
y_test  = keras_ml.to_categorical(y_test)

print('After conversion to categorical one-hot encoded labels: ', y_train[0])

# Create the model.
model = keras_ml.cnn_model()
model.summary()

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'],
             )

history = model.fit(X_train,
                    y_train,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_split=.3,
                   )


# Retrieve training results.
train_loss = history.history["loss"]
train_acc  = history.history["accuracy"]
valid_loss = history.history["val_loss"]
valid_acc  = history.history["val_accuracy"]

keras_ml.plot_results([ train_loss, valid_loss ],
            ylabel="Loss",
            ylim = [0.0, 5.0],
            metric_name=["Training Loss", "Validation Loss"],
            color=["g", "b"]);

keras_ml.plot_results([ train_acc, valid_acc ],
            ylabel="Accuracy",
            ylim = [0.0, 1.0],
            metric_name=["Training Accuracy", "Validation Accuracy"],
            color=["g", "b"])

In [ ]:
#Build a Gender model

In [ ]:
#Build a Glasses

In [ ]:
#Build a ID model


In [ ]:
#Build a Facial expression